In [4]:
!nvidia-smi

Wed Apr  2 18:53:25 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 4000     On   | 00000000:12:00.0 Off |                  N/A |
| 30%   45C    P8     9W / 125W |      1MiB /  7982MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 4000     On   | 00000000:86:00.0 Off |                  N/A |
| 30%   

In [2]:
!kill 3012384            

In [3]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
import nibabel as nib
import cv2
from tensorflow.keras import backend as K
import gc
tf.keras.backend.clear_session()
gc.collect()

609

In [5]:
def resize_slice(slice, target_shape):
    return cv2.resize(slice, target_shape, interpolation=cv2.INTER_LINEAR)

def load_patient_slices(patient_folder, num_classes=4):
    images, masks = [], []
    patient_data = []

    for phase in ["ED", "ES"]:
        image_path = os.path.join(patient_folder, f"patient{patient_folder[-3:]}_{phase}_processed.nii.gz")
        mask_path = os.path.join(patient_folder, f"patient{patient_folder[-3:]}_{phase}_gt_processed.nii.gz")

        if not os.path.exists(image_path) or not os.path.exists(mask_path):
            print(f"⚠️ Missing file: {image_path} or {mask_path}")
            continue  

        image_nii = nib.load(image_path)
        mask_nii = nib.load(mask_path)

        image_array = image_nii.get_fdata()
        mask_array = mask_nii.get_fdata()

        image_array = (image_array - np.mean(image_array)) / (np.std(image_array) + 1e-10)

        phase_images, phase_masks = [], []
        for i in range(image_array.shape[2]): 
            img_slice = image_array[:, :, i]
            mask_slice = mask_array[:, :, i]
            if img_slice.shape != (224, 224):
                img_slice = resize_slice(img_slice, (224, 224))
            if mask_slice.shape != (224, 224):
                mask_slice = resize_slice(mask_slice, (224, 224))
            img_slice = np.expand_dims(img_slice, axis=-1)
            mask_slice = to_categorical(mask_slice, num_classes=num_classes).astype("float32")
            phase_images.append(img_slice)
            phase_masks.append(mask_slice)
        
        patient_data.append({
            'patient_id': patient_folder[-3:],
            'phase': phase,
            'images': np.array(phase_images, dtype=np.float32),
            'masks': np.array(phase_masks, dtype=np.float32)
        })

    return patient_data

def load_dataset_2d(root_dir, dataset_type):
    dataset = []
    dataset_path = os.path.join(root_dir, dataset_type)

    for patient_folder in sorted(os.listdir(dataset_path)):
        patient_path = os.path.join(dataset_path, patient_folder)
        if os.path.isdir(patient_path):
            patient_data = load_patient_slices(patient_path)
            dataset.extend(patient_data)
            print(f"✅ Loaded {len(patient_data)} phases from {patient_folder}")
    
    return dataset

data_2d = load_dataset_2d(r"/home/student11/ds/mlmed-huyproman/cropped_slices_processed_320", "testing")

✅ Loaded 2 phases from patient101
✅ Loaded 2 phases from patient102
✅ Loaded 2 phases from patient103
✅ Loaded 2 phases from patient104
✅ Loaded 2 phases from patient105
✅ Loaded 2 phases from patient106
✅ Loaded 2 phases from patient107
✅ Loaded 2 phases from patient108
✅ Loaded 2 phases from patient109
✅ Loaded 2 phases from patient110
✅ Loaded 2 phases from patient111
✅ Loaded 2 phases from patient112
✅ Loaded 2 phases from patient113
✅ Loaded 2 phases from patient114
✅ Loaded 2 phases from patient115
✅ Loaded 2 phases from patient116
✅ Loaded 2 phases from patient117
✅ Loaded 2 phases from patient118
✅ Loaded 2 phases from patient119
✅ Loaded 2 phases from patient120
✅ Loaded 2 phases from patient121
✅ Loaded 2 phases from patient122
✅ Loaded 2 phases from patient123
✅ Loaded 2 phases from patient124
✅ Loaded 2 phases from patient125
✅ Loaded 2 phases from patient126
✅ Loaded 2 phases from patient127
✅ Loaded 2 phases from patient128
✅ Loaded 2 phases from patient129
✅ Loaded 2 pha

In [6]:
def crop_or_pad_volume(volume, target_shape=(224, 224, 10)):
    z, x, y = volume.shape
    z_target, x_target, y_target = target_shape

    if z < z_target:
        pad_z = z_target - z
        pad_before = pad_z // 2
        pad_after = pad_z - pad_before
        volume = np.pad(volume, ((pad_before, pad_after), (0, 0), (0, 0)), mode='constant')
    elif z > z_target:
        start_z = (z - z_target) // 2
        volume = volume[start_z:start_z + z_target, :, :]
    
    if x < x_target or y < y_target:
        volume = np.pad(volume, ((0, 0), (0, x_target - x), (0, y_target - y)), mode='constant')
    elif x > x_target or y > y_target:
        start_x = (x - x_target) // 2
        start_y = (y - y_target) // 2
        volume = volume[:, start_x:start_x + x_target, start_y:start_y + y_target]

    return volume

def load_patient_volume(patient_folder, num_classes=4):
    patient_data = []

    for phase in ["ED", "ES"]:
        image_path = os.path.join(patient_folder, f"patient{patient_folder[-3:]}_{phase}_processed.nii.gz")
        mask_path = os.path.join(patient_folder, f"patient{patient_folder[-3:]}_{phase}_gt_processed.nii.gz")

        if not os.path.exists(image_path) or not os.path.exists(mask_path):
            print(f"⚠️ Missing file: {image_path} or {mask_path}")
            continue  

        image_nii = nib.load(image_path)
        mask_nii = nib.load(mask_path)

        image_array = image_nii.get_fdata()
        mask_array = mask_nii.get_fdata()

        original_z = image_array.shape[2]  # Store original number of slices
        image_array = (image_array - np.mean(image_array)) / (np.std(image_array) + 1e-10)

        image_array = crop_or_pad_volume(image_array)
        mask_array = crop_or_pad_volume(mask_array)

        image_array = np.expand_dims(image_array, axis=-1)
        mask_array = to_categorical(mask_array, num_classes=num_classes).astype("float32")

        patient_data.append({
            'patient_id': patient_folder[-3:],
            'phase': phase,
            'image': image_array,
            'mask': mask_array,
            'original_z': original_z
        })

    return patient_data

def load_dataset_3d(root_dir, dataset_type):
    dataset = []
    dataset_path = os.path.join(root_dir, dataset_type)

    for patient_folder in sorted(os.listdir(dataset_path)):
        patient_path = os.path.join(dataset_path, patient_folder)
        if os.path.isdir(patient_path):
            patient_data = load_patient_volume(patient_path)
            dataset.extend(patient_data)
            print(f"✅ Loaded {len(patient_data)} volumes from {patient_folder}")
    
    return dataset

data_3d = load_dataset_3d(r"/home/student11/ds/mlmed-huyproman/cropped_slices_processed_320", "testing")

✅ Loaded 2 volumes from patient101
✅ Loaded 2 volumes from patient102
✅ Loaded 2 volumes from patient103
✅ Loaded 2 volumes from patient104
✅ Loaded 2 volumes from patient105
✅ Loaded 2 volumes from patient106
✅ Loaded 2 volumes from patient107
✅ Loaded 2 volumes from patient108
✅ Loaded 2 volumes from patient109
✅ Loaded 2 volumes from patient110
✅ Loaded 2 volumes from patient111
✅ Loaded 2 volumes from patient112
✅ Loaded 2 volumes from patient113
✅ Loaded 2 volumes from patient114
✅ Loaded 2 volumes from patient115
✅ Loaded 2 volumes from patient116
✅ Loaded 2 volumes from patient117
✅ Loaded 2 volumes from patient118
✅ Loaded 2 volumes from patient119
✅ Loaded 2 volumes from patient120
✅ Loaded 2 volumes from patient121
✅ Loaded 2 volumes from patient122
✅ Loaded 2 volumes from patient123
✅ Loaded 2 volumes from patient124
✅ Loaded 2 volumes from patient125
✅ Loaded 2 volumes from patient126
✅ Loaded 2 volumes from patient127
✅ Loaded 2 volumes from patient128
✅ Loaded 2 volumes f

In [7]:
mismatches = []

for d2, d3 in zip(data_2d, data_3d):
    slices_2d = d2['images'].shape[0]
    slices_3d = d3['original_z']
    if slices_2d != slices_3d:
        mismatches.append((d2['patient_id'], d2['phase'], slices_2d, slices_3d))

if mismatches:
    print("❌ Slice count mismatches found:")
    for patient_id, phase, slices_2d, slices_3d in mismatches:
        print(f"Patient {patient_id}, Phase {phase}: 2D = {slices_2d}, 3D = {slices_3d}")
else:
    print("✅ All good. No mismatches.")

✅ All good. No mismatches.


In [8]:
def create_tf_dataset_2d(patient_data, batch_size):
    images = np.concatenate([p['images'] for p in patient_data], axis=0)
    masks = np.concatenate([p['masks'] for p in patient_data], axis=0)
    dataset = tf.data.Dataset.from_tensor_slices((images, masks))
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

def create_tf_dataset_3d(patient_data, batch_size):
    images = np.array([p['image'] for p in patient_data], dtype=np.float32)
    masks = np.array([p['mask'] for p in patient_data], dtype=np.float32)
    dataset = tf.data.Dataset.from_tensor_slices((images, masks))
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    test_dataset_2d = create_tf_dataset_2d(data_2d, batch_size=1)
    test_dataset_3d = create_tf_dataset_3d(data_3d, batch_size=1)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2025-04-02 18:53:37.371307: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6653 MB memory:  -> device: 0, name: Quadro RTX 4000, pci bus id: 0000:12:00.0, compute capability: 7.5
2025-04-02 18:53:37.372566: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 6653 MB memory:  -> device: 1, name: Quadro RTX 4000, pci bus id: 0000:86:00.0, compute capability: 7.5


In [9]:
del data_2d

In [10]:
def dice_loss_per_class(y_true, y_pred, num_classes=4, smooth=1e-6):
    dice_scores = []
    
    for i in range(1, num_classes):
        y_true_class = K.flatten(y_true[..., i])
        y_pred_class = K.flatten(y_pred[..., i])

        intersection = K.sum(y_true_class * y_pred_class)
        dice = (2. * intersection + smooth) / (K.sum(y_true_class) + K.sum(y_pred_class) + smooth)
        dice_scores.append(dice)

    return 1 - K.mean(K.stack(dice_scores), axis=0)  

def dice_score_per_class(y_true, y_pred, num_classes=4, smooth=1e-6):
    dice_scores = []
    
    for i in range(1, num_classes):
        y_true_class = K.flatten(y_true[..., i])
        y_pred_class = K.flatten(y_pred[..., i])

        intersection = K.sum(y_true_class * y_pred_class)
        dice = (2. * intersection + smooth) / (K.sum(y_true_class) + K.sum(y_pred_class) + smooth)
        dice_scores.append(dice)

    return K.mean(K.stack(dice_scores), axis=0)  

def dice_metric(y_true, y_pred):
    return dice_score_per_class(y_true, y_pred, num_classes=4)

def dice_score_each_individual_class(y_true, y_pred, num_classes=4, smooth=1e-6):
    dice_scores = []

    for i in range(num_classes):
        y_true_class = tf.keras.backend.flatten(y_true[..., i])
        y_pred_class = tf.keras.backend.flatten(y_pred[..., i])

        intersection = tf.keras.backend.sum(y_true_class * y_pred_class)
        dice = (2. * intersection + smooth) / (
            tf.keras.backend.sum(y_true_class) + tf.keras.backend.sum(y_pred_class) + smooth
        )
        dice_scores.append(dice)

    return dice_scores  

In [11]:
def dice_loss_per_class_3d(y_true, y_pred, num_classes=4, smooth=1e-6):
    dice_scores = []
    for i in range(1, num_classes):
        y_true_class = K.flatten(y_true[..., i])  
        y_pred_class = K.flatten(y_pred[..., i])
        intersection = K.sum(y_true_class * y_pred_class)
        dice = (2. * intersection + smooth) / (K.sum(y_true_class) + K.sum(y_pred_class) + smooth)
        dice_scores.append(dice)
    mean_dice = K.mean(K.stack(dice_scores), axis=0)
    return 1 - mean_dice
def dice_score_per_class_3d(y_true, y_pred, num_classes=4, smooth=1e-6):
    dice_scores = []
    for i in range(1, num_classes):  
        y_true_class = K.flatten(y_true[..., i])
        y_pred_class = K.flatten(y_pred[..., i])
        intersection = K.sum(y_true_class * y_pred_class)
        dice = (2. * intersection + smooth) / (K.sum(y_true_class) + K.sum(y_pred_class) + smooth)
        dice_scores.append(dice)
    return K.mean(K.stack(dice_scores), axis=0)  

In [12]:
model_2d = tf.keras.models.load_model(r"/home/student11/ds/mlmed-huyproman/models/best_unet2D_320_again_1",
                                      custom_objects={
                                    'dice_loss_per_class': dice_loss_per_class,
                                    'dice_metric': dice_metric
                                })

In [13]:
model_3d = tf.keras.models.load_model(r"/home/student11/ds/mlmed-huyproman/models/best_unet3D_224x224x10",
                                      custom_objects={
                                    'dice_loss_per_class_3d': dice_loss_per_class_3d,
                                    'dice_score_per_class_3d': dice_score_per_class_3d
                                })

In [14]:
def get_predictions(model, dataset):
    predictions = []
    for x, _ in dataset:
        pred = model.predict(x, verbose=0)
        if isinstance(pred, dict):
            pred = list(pred.values())[0]
        predictions.append(pred)
    return tf.concat(predictions, axis=0)

y_pred_2d = get_predictions(model_2d, test_dataset_2d)  

2025-04-02 18:53:46.443688: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2025-04-02 18:53:46.999380: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [15]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    y_pred_3d = []
    for batch in test_dataset_3d:
        images, _ = batch
        pred = model_3d.predict(images, verbose=0)
        main_pred = pred[0]  
        y_pred_3d.append(main_pred)

y_pred_3d = tf.concat(y_pred_3d, axis=0)  

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2025-04-02 18:54:41.012088: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 5.97GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


In [16]:
y_true_2d = []
for _, y in test_dataset_2d:
    y_true_2d.append(y)
y_true_2d = tf.concat(y_true_2d, axis=0)

# y_true_3d = []
# for batch in test_dataset_3d:
#     _, masks = batch  
#     y_true_3d.append(masks[0]) 

In [17]:
filtered_preds = []

for i, patient in enumerate(data_3d):
    image = patient["image"]  
    pred = y_pred_3d[i]    

    for z in range(image.shape[2]):
        img_slice = image[:, :, z, 0]
        if not tf.reduce_all(img_slice == 0):
            pred_slice = pred[:, :, z, :]
            filtered_preds.append(pred_slice)

y_pred_3d_flat = tf.stack(filtered_preds)

In [18]:
fused_predictions = []
current_3d_idx = 0

for vol_data in data_3d:
    original_z = vol_data['original_z']
    y_pred_2d_valid = y_pred_2d[current_3d_idx : current_3d_idx + original_z]
    y_pred_3d_valid = y_pred_3d_flat[current_3d_idx : current_3d_idx + original_z]
    fused_pred = y_pred_2d_valid * 0.6 + y_pred_3d_valid * 0.4
    fused_predictions.append(fused_pred)
    current_3d_idx += original_z

fused_predictions = tf.concat(fused_predictions, axis=0)

In [19]:
y_pred_labels = tf.argmax(fused_predictions, axis=-1)  
y_true_labels = tf.argmax(y_true_2d, axis=-1)  

def dice_score_per_class(y_true, y_pred, num_classes=4, smooth=1e-6):
    dice_scores = []
    for c in range(1, num_classes):
        y_true_c = tf.cast(y_true == c, tf.float32)
        y_pred_c = tf.cast(y_pred == c, tf.float32)
        intersection = tf.reduce_sum(y_true_c * y_pred_c)
        union = tf.reduce_sum(y_true_c) + tf.reduce_sum(y_pred_c)
        dice = (2. * intersection + smooth) / (union + smooth)
        dice_scores.append(dice)
    return dice_scores

with strategy.scope():
    dice_scores = dice_score_per_class(y_true_labels, y_pred_labels)

    for c, score in enumerate(dice_scores):
        print(f"Dice score for class {c+1}: {score.numpy()*100:.2f}%")

    average_main = tf.reduce_mean(dice_scores[1:4])
    print(f"Average Dice score for main classes: {average_main.numpy()*100:.2f}%")

Dice score for class 1: 90.85%
Dice score for class 2: 87.91%
Dice score for class 3: 95.24%
Average Dice score for main classes: 91.57%


fucking dick urkel piece of shit cái trên này fix mãi đ đc

---
# FINAL RESULTS

## 0.6 - 0.4
* Dice score for class 1: 90.85%

* Dice score for class 2: 87.91%

* Dice score for class 3: 95.24%

=>Average Dice score for main classes: 91.57%